In [57]:
import calendar
import json
import requests
import csv

filename = "output.csv"

finraUrl = "https://api.finra.org/data/group/OTCMarket/name/weeklySummary"

header = 'issueSymbolIdentifier,issueName,totalWeeklyShareQuantity,totalWeeklyTradeCount,marketParticipantName,tierIdentifier,weekStartDate\n'

with open(filename, "w") as f:
    f.seek(0)
    f.write(header)
    f.truncate()

for year in [2019,2020,2021]:
    for month in range(1, 12):
        startDate = str(year) + "-" + str(month).zfill(2) + "-" + "01"
        endDate = str(year) + "-" + str(month).zfill(2) + "-" + str(calendar._monthlen(year, month))
        print(startDate)
        print(endDate)

        tempQuery = {
        "fields":["issueSymbolIdentifier", "issueName", "totalWeeklyShareQuantity","totalWeeklyTradeCount","marketParticipantName","tierIdentifier","weekStartDate"],  
        "limit" : 50000,
        "domainFilters": [{
        "fieldName" : "issueSymbolIdentifier",
        "values" : [ "GME", "AMC", "KOSS", "NAKD", "BBBY", "NOK", "GM", "AAPL", "TSLA", "MSFT" ]
        }],
        "dateRangeFilters": [ {

            "startDate" : startDate,

            "endDate" : endDate,

            "fieldName" : "weekStartDate"

        } ]
        }

        query = json.dumps(tempQuery)

        r = requests.post(finraUrl, query)
        with open(filename, 'a') as f:
            reader = csv.DictReader(r.text.splitlines(), delimiter=',')
            writer = csv.DictWriter(f, reader.fieldnames)
            writer.writerows(reader)



2019-01-01
2019-01-31
2019-02-01
2019-02-28
2019-03-01
2019-03-31
2019-04-01
2019-04-30
2019-05-01
2019-05-31
2019-06-01
2019-06-30
2019-07-01
2019-07-31
2019-08-01
2019-08-31
2019-09-01
2019-09-30
2019-10-01
2019-10-31
2019-11-01
2019-11-30
2020-01-01
2020-01-31
2020-02-01
2020-02-29
2020-03-01
2020-03-31
2020-04-01
2020-04-30
2020-05-01
2020-05-31
2020-06-01
2020-06-30
2020-07-01
2020-07-31
2020-08-01
2020-08-31
2020-09-01
2020-09-30
2020-10-01
2020-10-31
2020-11-01
2020-11-30
2021-01-01
2021-01-31
2021-02-01
2021-02-28
2021-03-01
2021-03-31
2021-04-01
2021-04-30
2021-05-01
2021-05-31
2021-06-01
2021-06-30
2021-07-01
2021-07-31
2021-08-01
2021-08-31
2021-09-01
2021-09-30
2021-10-01
2021-10-31
2021-11-01
2021-11-30


In [2]:
import yfinance as yf
data = yf.download(

        tickers = ["GME", "AMC", "KOSS", "NAKD", "BBBY", "NOK", "GM", "AAPL", "TSLA", "MSFT"],

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "1y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True

)

print(data)

[*********************100%***********************]  10 of 10 completed
                 BBBY                                                     GM  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2016-06-03  38.999544  39.025700  38.179971  38.755417    1629500  25.405547   
2016-06-06  38.799017  39.548842  38.380511  39.522686    2593300  25.202913   
2016-06-07  39.670899  39.862716  39.313425  39.601147    1626800  25.464650   
2016-06-08  39.758080  40.002211  39.330857  39.697048    1257800  25.652013   
2016-06-09  39.357008  39.374446  38.118929  38.363056    2288400  25.207382   
...               ...        ...        ...        ...        ...        ...   
2021-05-27  26.420000  28.120001  25.309999  27.010000    9701700  60.000000   
2021-05-28  27.320000  30.219999  27.219999  27.990000   13567700  59.880001   
2021-06-01  28.950001  29.240000  27.230000  27.2